In [1]:
# The Usual
import pandas as pd

#NLTK
import nltk; nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
import pyfpgrowth
# FP-Growth
# from mlxtend import frequent_patterns
# from mlxtend.preprocessing import TransactionEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\uyent\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\uyent\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [4]:
df = pd.merge(listings, reviews, on='listing_id')

In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['name']))


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)


In [9]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [19]:
# te = TransactionEncoder()
# te_ary = te.fit(data_words_bigrams).transform(data_words_bigrams)
# df = pd.DataFrame(te_ary, columns=te.columns_)
# df

patterns= pyfpgrowth.find_frequent_patterns(data_words_nostops, 10000)
patterns

{('full',): 10108,
 ('stylish',): 10408,
 ('backyard',): 10417,
 ('prospect',): 10440,
 ('sq',): 10495,
 ('newly',): 10582,
 ('steps',): 10719,
 ('beds',): 11331,
 ('hill',): 11581,
 ('entrance',): 11604,
 ('entrance', 'private'): 11683,
 ('nice',): 11629,
 ('perfect',): 11636,
 ('oasis',): 11646,
 ('gorgeous',): 11753,
 ('soho',): 11811,
 ('queen',): 12036,
 ('lower',): 12060,
 ('east', 'lower'): 10655,
 ('ny',): 12289,
 ('two',): 12733,
 ('kitchen',): 13172,
 ('astoria',): 13182,
 ('place',): 13267,
 ('queens',): 13349,
 ('artist',): 13385,
 ('airport',): 13412,
 ('bd',): 13890,
 ('entire',): 14278,
 ('york',): 14629,
 ('new', 'york'): 14645,
 ('view',): 14802,
 ('chelsea',): 14847,
 ('big',): 15067,
 ('amazing',): 15184,
 ('historic',): 15198,
 ('bdrm',): 15206,
 ('away',): 15416,
 ('away', 'home'): 16556,
 ('heights',): 15807,
 ('minutes',): 16592,
 ('townhouse',): 17108,
 ('comfortable',): 17130,
 ('slope',): 17676,
 ('park', 'slope'): 16980,
 ('huge',): 18346,
 ('space',): 18380,

In [20]:
# frequent_patterns.fpgrowth(df, min_support=0.4, use_colnames=True)

In [21]:
rules = pyfpgrowth.generate_association_rules(patterns, 0.7)
rules

{('entrance',): (('private',), 1.0068079972423303),
 ('lower',): (('east',), 0.8834991708126037),
 ('york',): (('new',), 1.0010937179574817),
 ('away',): (('home',), 1.0739491437467565),
 ('slope',): (('park',), 0.9606245756958588),
 ('bathroom',): (('private',), 0.9014533339771136),
 ('one',): (('bedroom',), 0.7113103710029415),
 ('side',): (('east',), 0.7370168971935331),
 ('square',): (('times',), 0.7061963477446542),
 ('times',): (('square',), 0.7987501324012287),
 ('central', 'near'): (('park',), 0.9339267688354593)}